## INPUT DEL MODELO

In [1]:
PONDERADOR_PRECIO = 1
ponderador_costo_mina = 0.5
ponderador_costo_planta_base = 1

precio_base = 3791.912 * PONDERADOR_PRECIO

ZZ = 530

x_iii = 490
y_iii = 580
x_fff = 780
y_fff = 920

## RUTA DONDE SE ALMACENAN Y EXPORTAN LOS RESULTADOS

In [2]:
RUTA_PC = 'C:/Users/Ignacio/Desktop/ESCENARIOS/ESCENARIOS SUBTE/RESULTADOS PRECIO'
RUTA_RESULTADOS_T1 = '/T1'

#ESCENARIO_ACTUAL = '/PRECIO' + str(int(PONDERADOR_PRECIO*100)) + '%'
ESCENARIO_ACTUAL = '/CM S' + str(int(PONDERADOR_C_MINA*100)) + '%'
#ESCENARIO_ACTUAL = '/CPLANTA' + str(int(PONDERADOR_C_PLANTA*100)) + '%'
#ESCENARIO_ACTUAL = '/ESCENARIO BASE'

STRING_OPCION = '/' + str(ZZ)

## LIBRERIAS 
(NO SE USAN TODAS, TENGO DE ARRASTRE DEL MODELO ANTERIOR DONDE SE PLOTEAN LOS RESULTADOS)

In [3]:
import pandas     as      pd
import time
import os
import xlsxwriter
from   pathlib    import Path
import gurobipy   as     gp
from   gurobipy   import GRB

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

from openpyxl import Workbook



from Modulo_Globales import *
from CA_BloqueSuperior import *
from Funcion_Drawpoint import *
from Visualizacion_Global import *
from Preparacion_CA import *
from Preparacion_S import *
from Preparacion_Integrado import *
from FUNCION_MODELOS_DESINTEGRADOS import * 

## CARGAR LOS ARCHIVOS EXCEL

In [4]:
## Se debe seleccionar la ruta del archivo, con su nombre y extensión
data_folder = Path('C:/Users/Ignacio/Desktop/ESCENARIOS/MODELO DE BLOQUES')
Excel_Rebloqueado = 'Modelo_F_OG_4_4_4.xlsx'
Excel_General = 'Modelo_F_OG.xlsx'

## Se crea un dataframe del arcchivo que se lee
# Cielo Abierto
df_C = pd.read_excel(data_folder/Excel_Rebloqueado, engine="openpyxl")
# Subterráneo
df_S = pd.read_excel(data_folder/Excel_General, engine="openpyxl")


cord_x_C = df_C['X'].to_dict()
cord_y_C = df_C['Y'].to_dict()
cord_z_C = df_C['Z'].to_dict()
TON_C = df_C['Ton'].to_dict()
MIN_C = df_C['Mineral'].to_dict()
REC_C = df_C['Recuperación'] .to_dict()
LEY_C = df_C['%Cu'].to_dict()
CM_CA_C = df_C['CMina CA'].to_dict()
CP_CA_C = df_C['CPlanta CA'].to_dict()

## Base de datos del modelo subterráneo
cord_x = df_S['X'].to_dict()             
cord_y = df_S['Y'].to_dict()             
cord_z = df_S['Z'].to_dict()             
TON = df_S['Ton'].to_dict()              
MIN = df_S['Mineral'].to_dict()          
REC = df_S['Recuperación'].to_dict()     
LEY = df_S['%Cu'].to_dict()
CM_CA = df_S['CMina CA'].to_dict()      
CP_CA = df_S['CPlanta CA'].to_dict()     
CM_S = df_S['CMINA S'].to_dict()         
CP_S = df_S['CPlanta S'].to_dict()

limites_x = t_cord(cord_x)
limites_y = t_cord(cord_y)
limites_z = t_cord(cord_z)

limites_x_C = t_cord(cord_x_C)
limites_y_C = t_cord(cord_y_C)
limites_z_C = t_cord(cord_z_C)

rebloqueo_x = limites_x_C[0]//limites_x[0]
rebloqueo_y = limites_y_C[0]//limites_y[0]
rebloqueo_z = limites_z_C[0]//limites_z[0]

bloques_CA = list()
bloques_S = list()

cantidad_bloques(cord_x_C, bloques_CA)
cantidad_bloques(cord_x, bloques_S)

bloques_usados = bloques_CA.copy()
block_sup = bloque_final(bloques_usados, limites_x_C, limites_y_C, limites_z_C)

predecessor_block = []
for i in range(len(bloques_usados)):
    for j in block_sup[i]:
        aux_1 = []
        aux_1.append(bloques_usados[i])
        aux_1.append(j)
        predecessor_block.append(aux_1)

def b_sup_init():
    return (i for i in range(len(predecessor_block)))

modelo_general = Escalado_conjunto_posicion (limites_x,limites_y,limites_z,limites_x_C,limites_y_C,limites_z_C)
# modelo_general[0] es de la forma (x_min, y_min, z_min, x_max, y_max, z_max)
distancia_no_rebloq = modelo_general[1]  # Se debe dividir por el tamaño del bloque no rebloqueado
distancia_rebloq = modelo_general[2]     # Se debe dividir por el tamaño del bloque no rebloqueado
bloques_total = modelo_general[3]


matriz = matriz_global(bloques_total)

## CALCULOS DEL MODELO

In [6]:
cota_seg = 30
dif_centroide = limites_x_C[0]//2 - limites_x[0]//2

t_C_2 = {0:1}
RMu_t_2 = {0:99999999999999}
RMl_t_2 = {0:0}
RPu_t_2 = {0:99999999999999}
RPl_t_2 = {0:0}
qu_t_2 = {0:3}
ql_t_2 = {0:0}

time_max_C_2 = t_C_2[max(t_C_2)]

t_S_2 = {0:1}
MU_mt_2 = {0:99999999999999}
ML_mt_2 = {0:0}
MU_pt_2 = {0:99999999999999}
ML_pt_2 = {0:0}
qU_dt_2 = {0: 1}
qL_dt_2 = {0: 0}
A_d_2   = {0: 1}
NU_nt_2 = {0: 99999999}
NL_nt_2 = {0: 0}
N_t_2   = {0: 99999999}
RL_dt_2 = {0: 0.8}
RU_dt_2 = {0: 1}

time_max_S_2 = t_S_2[max(t_S_2)]

E1 = [t_C_2,RMu_t_2,RMl_t_2,RPu_t_2,RPl_t_2,qu_t_2,ql_t_2,time_max_C_2,t_S_2,MU_mt_2,ML_mt_2,MU_pt_2,
                          ML_pt_2,qU_dt_2,qL_dt_2,A_d_2,NU_nt_2,NL_nt_2,N_t_2,RL_dt_2,RU_dt_2,time_max_S_2]

col_height = 300
DP_init = 0       #### Tipo de extracción

## ACÁ INICIA LA ITERACIÓN

In [7]:
lista_resultados = []



print('|Parámetro|Min|Max|SALTO|ESCENARIOS|')
print('|:--:|:--:|:--:|:--:|:--:|')
print('| X_i |'+str(x_iii-30)+' | '+str(x_iii+50) + ' | 10 |2|')
print('| Y_i |'+str(y_iii-30)+' | '+str(y_iii+50) + ' | 10 |2|')
print('| X_f |'+str(x_fff-50)+' | '+str(x_fff+30) + ' | 10 |2|')
print('| Y_f |'+str(y_fff-50)+' | '+str(y_fff+30) + ' | 10 |2|')

print('\n\n')

for x in range(x_iii-30,x_iii+50,10):
    for y in range(y_iii-30,y_iii+50,10):
        pos_x = x
        pos_y = y
        pos_z = ZZ
        for x_2 in range(x_fff-50,x_fff+30,10):
            for y_2 in range(y_fff-50,y_fff+30,10):
                print(str(x)+' | '+str(y)+' | '+str(pos_z)+' | '+str(x_2)+' | '+str(y_2))
                pos_x_f = x_2
                pos_y_f = y_2

                drawpoint, TON_d, MIN_D,LEY_D, C_P_D, C_M_D, predecessor, x_draw, y_draw, z_draw = Funcion_drawpoint(
                    pos_x, pos_y, pos_z, col_height, DP_init, limites_x, limites_y, limites_z, TON, CP_S, CM_S, MIN, 
                    LEY, pos_x_f, pos_y_f, 0)



                RESULTADOS_S = FUNCION_SUBTE(drawpoint,col_height,predecessor,
                         TON_d,MIN_D,REC,LEY_D,C_P_D,C_M_D,
                         E1[8],E1[9],E1[10],E1[11],E1[12],E1[13],E1[14],E1[15],E1[16],E1[17],E1[18],E1[19],E1[20],E1[21],
                         0.01,precio_base,ponderador_costo_mina,ponderador_costo_planta_base)

                try:
                    lista_aux = [RESULTADOS_S[0],x,y,pos_z,x_2,y_2,RESULTADOS_S[2],sum(TON_d),sum(MIN_D)]
                    lista_resultados.append(lista_aux)
                except:
                    pass

lista_resultados.sort(reverse = True)
### Calculo de estadisticas de los mejores resultados
x_lista = []
y_lista = []
z_lista = []
x_f_lista = []
y_f_lista = []

for i in lista_resultados[:int(len(lista_resultados)*0.1)]:
    x_lista.append(i[1])
    y_lista.append(i[2])
    z_lista.append(i[3])
    x_f_lista.append(i[4])
    y_f_lista.append(i[5])

x_lista.sort()
y_lista.sort()
z_lista.sort()
x_f_lista.sort()
y_f_lista.sort()

|Parámetro|Min|Max|SALTO|ESCENARIOS|
|:--:|:--:|:--:|:--:|:--:|
| X_i |460 | 540 | 10 |2|
| Y_i |550 | 630 | 10 |2|
| X_f |730 | 810 | 10 |2|
| Y_f |870 | 950 | 10 |2|



460 | 550 | 530 | 730 | 870
Academic license - for non-commercial use only - expires 2021-08-21
Using license file C:\Users\Ignacio\gurobi.lic
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1242 rows, 462 columns and 3540 nonzeros
Model fingerprint: 0xb03d8365
Model has 154 quadratic constraints
Variable types: 154 continuous, 308 integer (308 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e+04, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range     

Variable types: 168 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [5e+04, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1354 rows and 504 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 20 available processors)

Solution count 1: 3.62058e+08 

Optimal solution found (tolerance 1.00e-02)
Best objective 3.620580456083e+08, best bound 3.620580456083e+08, gap 0.0000%
460 | 550 | 530 | 730 | 930
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9

Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1354 rows, 504 columns and 3862 nonzeros
Model fingerprint: 0xa0ce0532
Model has 168 quadratic constraints
Variable types: 168 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [5e+04, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1354 rows and 504 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 se

Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 20 available processors)

Solution count 1: 3.61157e+08 

Optimal solution found (tolerance 1.00e-02)
Best objective 3.611565481423e+08, best bound 3.611565481423e+08, gap 0.0000%
460 | 550 | 530 | 750 | 880
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1330 rows, 495 columns and 3793 nonzeros
Model fingerprint: 0x73ebc62b
Model has 165 quadratic constraints
Variable types: 165 continuous, 330 integer (330 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e+04, 6e+06]
  Bounds range     [1e+00, 1e+00]
  R

Variable types: 195 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [5e+04, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1570 rows and 585 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 20 available processors)

Solution count 1: 4.07208e+08 

Optimal solution found (tolerance 1.00e-02)
Best objective 4.072079861497e+08, best bound 4.072079861497e+08, gap 0.0000%
460 | 550 | 530 | 750 | 940
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9

...

   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 874 rows, 324 columns and 2482 nonzeros
Model fingerprint: 0x4cbcc9f7
Model has 108 quadratic constraints
Variable types: 108 continuous, 216 integer (216 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+06, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 874 rows and 324 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 20 available processors)

Solution count 1: 3.98072e+08 

Optimal solution found (tolerance 1.00e-02)
Best objective


Optimal solution found (tolerance 1.00e-02)
Best objective 4.679579869185e+08, best bound 4.679579869185e+08, gap 0.0000%
530 | 590 | 530 | 770 | 930
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1066 rows, 396 columns and 3034 nonzeros
Model fingerprint: 0xcd6eec65
Model has 132 quadratic constraints
Variable types: 132 continuous, 264 integer (264 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+06, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Pre

         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1154 rows and 429 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 20 available processors)

Solution count 1: 4.61995e+08 

Optimal solution found (tolerance 1.00e-02)
Best objective 4.619951469509e+08, best bound 4.619951469509e+08, gap 0.0000%
530 | 620 | 530 | 790 | 870
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 842 rows, 312 columns and 2390 nonzeros
Model fingerprint: 0x93b98145
Model has 104 quadratic constraints
Variable types: 104 continuous, 208 integer (208 

Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1050 rows, 390 columns and 2988 nonzeros
Model fingerprint: 0x0cd230ce
Model has 130 quadratic constraints
Variable types: 130 continuous, 260 integer (260 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [7e+05, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1050 rows and 390 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 20 available processors)

Solution count 1: 4.60554e+08 

Optimal solution found (tolerance 1.00e-02)
Best objective 4.605540895587e+08, best bound 4.605540895587e+08, gap 0.0000%
530 | 620 | 530 | 790 | 930
Changed 

530 | 620 | 530 | 800 | 900
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 36000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 1018 rows, 378 columns and 2896 nonzeros
Model fingerprint: 0xb76d998f
Model has 126 quadratic constraints
Variable types: 126 continuous, 252 integer (252 binary)
Coefficient statistics:
  Matrix range     [8e-01, 5e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e+05, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+14]
  QRHS range       [8e-01, 8e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1018 rows and 378 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 si